# Data Analysis and Visualization Project (Source Data)
## Charting the Path to Decent Work

This notebook performs "on-the-fly" joining of the original source datasets to minimize data sparseness and maximize the available information for analysis.

**Datasets:**
1. `education.csv`
2. `gender.csv`
3. `poverty.csv`
4. `social_protection_labor.csv`

**Methodology:**
We will load each dataset individually and merge them on `country` and `year`. Using an **outer join** strategy ensures we retain all available records, even if a country is missing data for a specific sector (e.g., education) in a given year.

**Visualizations:**
1. Global Unemployment Trends (1991-2023)
2. Economic Productivity by Region (2022)
3. Philippines Case Study: Services vs. Vulnerable Employment
4. Global Unemployment Rate Map (2022)
5. GDP per Person vs. Unemployment Rate (2022)
6. Correlation Matrix of Key Indicators

In [8]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pycountry
import pycountry_convert as pc
import warnings

# Suppress warnings
warnings.filterwarnings('ignore')

# 1. Load Data
print("Loading datasets...")
try:
    edu_df = pd.read_csv('education.csv')
    gen_df = pd.read_csv('gender.csv')
    pov_df = pd.read_csv('poverty.csv')
    soc_df = pd.read_csv('social_protection_labor.csv')
    print("Datasets loaded successfully.")
except FileNotFoundError as e:
    print(f"Error loading data: {e}")

# 2. Inspect Shapes
print(f"Education: {edu_df.shape}")
print(f"Gender: {gen_df.shape}")
print(f"Poverty: {pov_df.shape}")
print(f"Social Protection: {soc_df.shape}")

# 3. On-the-fly Joining
# We merge on 'country' and 'year'. 
# We use 'outer' join to keep all country-year combinations found in ANY file.
print("Merging datasets...")
df = soc_df.merge(edu_df, on=['country', 'year'], how='outer')
df = df.merge(gen_df, on=['country', 'year'], how='outer')
df = df.merge(pov_df, on=['country', 'year'], how='outer')

print(f"Merged Dataset Shape: {df.shape}")

# 4. Preprocessing
# Rename columns to Title Case for consistency with previous code
df.rename(columns={'country': 'Country', 'year': 'Year'}, inplace=True)

# Filter years (1991-2023)
df = df[(df['Year'] >= 1991) & (df['Year'] <= 2023)]

# Helper to get continent
def get_continent(alpha_3):
    try:
        country = pycountry.countries.get(alpha_3=alpha_3)
        if country:
            alpha_2 = country.alpha_2
            continent_code = pc.country_alpha2_to_continent_code(alpha_2)
            continent_name = pc.convert_continent_code_to_continent_name(continent_code)
            return continent_name
    except:
        return None
    return None

# Add Continent column
print("Mapping countries to continents...")
df['Continent'] = df['Country'].apply(get_continent)

# Filter for valid countries (remove aggregates if any, based on continent mapping)
countries_df = df[df['Continent'].notna()].copy()
print(f"Final Analysis Dataset: {len(countries_df)} records.")

# Define Column Constants (using the column names from the source files)
col_unemp_total_orig = 'Unemployment, total (% of total labor force) (modeled ILO estimate)'
col_unemp_youth_orig = 'Unemployment, youth total (% of total labor force ages 15-24) (modeled ILO estimate)'
col_gdp_orig = 'GDP per person employed (constant 2021 PPP $)'
col_vuln_orig = 'Vulnerable employment, total (% of total employment) (modeled ILO estimate)'
col_serv_orig = 'Employment in services (% of total employment) (modeled ILO estimate)'

# Check if columns exist
missing_cols = [c for c in [col_unemp_total_orig, col_unemp_youth_orig, col_gdp_orig, col_vuln_orig, col_serv_orig] if c not in countries_df.columns]
if missing_cols:
    print(f"Warning: The following columns are missing from the merged dataset: {missing_cols}")
else:
    print("All key indicator columns found.")

# Rename columns for cleaner visuals
rename_map = {
    col_unemp_total_orig: 'Total Unemployment (%)',
    col_unemp_youth_orig: 'Youth Unemployment (%)',
    col_gdp_orig: 'GDP per Person ($)',
    col_vuln_orig: 'Vulnerable Employment (%)',
    col_serv_orig: 'Services Employment (%)'
}

print("Renaming columns to shorter versions...")
df.rename(columns=rename_map, inplace=True)
countries_df.rename(columns=rename_map, inplace=True)

# Update variables to use new names
col_unemp_total = 'Total Unemployment (%)'
col_unemp_youth = 'Youth Unemployment (%)'
col_gdp = 'GDP per Person ($)'
col_vuln = 'Vulnerable Employment (%)'
col_serv = 'Services Employment (%)'

Loading datasets...
Datasets loaded successfully.
Education: (13560, 54)
Gender: (16884, 73)
Poverty: (2426, 8)
Social Protection: (9288, 32)
Merging datasets...
Merged Dataset Shape: (16884, 161)
Mapping countries to continents...
Final Analysis Dataset: 7029 records.
All key indicator columns found.
Renaming columns to shorter versions...
Merged Dataset Shape: (16884, 161)
Mapping countries to continents...
Final Analysis Dataset: 7029 records.
All key indicator columns found.
Renaming columns to shorter versions...


In [9]:
# --- Viz 1: Global Unemployment Trends (Time Series) ---
global_trends = countries_df.groupby('Year')[[col_unemp_total, col_unemp_youth]].mean().reset_index()

fig1 = go.Figure()
fig1.add_trace(go.Scatter(x=global_trends['Year'], y=global_trends[col_unemp_total],
                    mode='lines', name='Total Unemployment'))
fig1.add_trace(go.Scatter(x=global_trends['Year'], y=global_trends[col_unemp_youth],
                    mode='lines', name='Youth Unemployment'))
fig1.update_layout(title='Global Unemployment Trends (1991-2023)',
                   xaxis_title='Year',
                   yaxis_title='Unemployment Rate (%)',
                   legend_title='Indicator',
                   template='plotly_white')
# Add shading for economic events
fig1.add_vrect(x0=2008, x1=2009, fillcolor="gray", opacity=0.2, layer="below", line_width=0, annotation_text="2008 Crisis", annotation_position="top left")
fig1.add_vrect(x0=2020, x1=2021, fillcolor="gray", opacity=0.2, layer="below", line_width=0, annotation_text="COVID-19", annotation_position="top left")

fig1.show()

In [10]:
# --- Viz 2: Productivity by Region (Box Plot) ---
# Use 2022 data
df_2022 = countries_df[countries_df['Year'] == 2022]
fig2 = px.box(df_2022, x='Continent', y=col_gdp, 
              points="all", 
              title='Economic Productivity by Region (2022)',
              log_y=True,
              color='Continent',
              template='plotly_white')
fig2.show()

In [11]:
# --- Viz 3: Philippines Case Study (Dual Axis) ---
phl_df = df[df['Country'] == 'PHL'].sort_values('Year')
fig3 = make_subplots(specs=[[{"secondary_y": True}]])
fig3.add_trace(go.Scatter(x=phl_df['Year'], y=phl_df[col_serv],
                    mode='lines', name='Employment in Services'), secondary_y=False)
fig3.add_trace(go.Scatter(x=phl_df['Year'], y=phl_df[col_vuln],
                    mode='lines', name='Vulnerable Employment', line=dict(dash='dash')), secondary_y=True)
fig3.update_layout(title='Philippines: Services vs. Vulnerable Employment (1991-2023)',
                   xaxis_title='Year',
                   template='plotly_white')
fig3.update_yaxes(title_text="Employment in Services (%)", secondary_y=False)
fig3.update_yaxes(title_text="Vulnerable Employment (%)", secondary_y=True)
fig3.show()

In [12]:
# --- Viz 4: Choropleth Map (Unemployment 2022) ---
fig4 = px.choropleth(df_2022, locations="Country",
                    color=col_unemp_total,
                    hover_name="Country",
                    color_continuous_scale=px.colors.sequential.Plasma,
                    title="Global Unemployment Rate (2022)",
                    template='plotly_white')
fig4.show()

In [13]:
# --- Viz 5: Scatter Plot (GDP vs Unemployment) ---
fig5 = px.scatter(df_2022, x=col_gdp, 
                  y=col_unemp_total,
                  color="Continent",
                  hover_name="Country",
                  log_x=True,
                  title="GDP per Person Employed vs. Unemployment Rate (2022)",
                  template='plotly_white')
fig5.show()

In [14]:
# --- Viz 6: Correlation Heatmap ---
cols = [col_unemp_total, col_unemp_youth, col_gdp, col_vuln, col_serv]
# Rename columns for cleaner heatmap
cols_renamed = {
    col_unemp_total: 'Total Unemployment',
    col_unemp_youth: 'Youth Unemployment',
    col_gdp: 'GDP per Person',
    col_vuln: 'Vulnerable Emp.',
    col_serv: 'Services Emp.'
}
corr_df = countries_df[cols].rename(columns=cols_renamed)
corr_matrix = corr_df.corr()
fig6 = px.imshow(corr_matrix, text_auto=True, title="Correlation Matrix of Key Indicators",
                 color_continuous_scale='RdBu_r', origin='lower')
fig6.show()